In [1]:
%pylab notebook

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K

import primo.models
import primo.datasets
import primo.tools.sequences as seqtools

from copy import deepcopy

from primo.models.cas9_keras import log10_crispr_spec, log10_norm_crispr_spec, linear_crispr_spec, dotproduct_crispr_spec, dotproduct_linearized

from primo.models.encoder import entropy_regularizer

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-eso8dgl4 because the default path (/tf/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Populating the interactive namespace from numpy and matplotlib
1 Physical GPUs, 1 Logical GPUs


In [2]:
train_dataset = primo.datasets.OpenImagesTrain(
    '/tf/open_images/train/', switch_every=10**5
)
validation_dataset = primo.datasets.OpenImagesVal('/tf/open_images/validation/')

# To see how this value was derived, please consult the Materials and Methods subsection under Feature Extraction section.
similarity_threshold = 75
# Intuitively determined:
batch_size = 150
val_batch_size = 150


def keras_batch_generator(dataset_batch_generator):
    # Yield datasets
    while True:
        # This tuple contains:
        # indices: a positive integer uniquely identifying an image. This index is obtained by enumerating all the images in the dataset (before splitting them into test/train/validate datasets)
        # pairs:
        indices, triplets = next(dataset_batch_generator)
        yield triplets, np.zeros(len(triplets))

train_batch_generator = keras_batch_generator(
    primo.datasets.dataset.triplet_batch_generator(
        train_dataset.random_features(batch_size),
        similarity_threshold
    )
)

val_batch_generator = keras_batch_generator(
    primo.datasets.dataset.triplet_batch_generator(
        validation_dataset.random_features(val_batch_size),
        similarity_threshold
    )
)


train_inputs, train_targets = next(train_batch_generator)
val_inputs, val_targets = next(val_batch_generator)

switching to train_d.h5 and train_7.h5


In [3]:
OUTPUT_LEN = 20
GUMBEL_SOFTMAX_TEMP = 10.0

def sample_gumbel(shape, eps=1e-20):
    U = tf.random.uniform(shape, minval=0, maxval=1)
    return -tf.math.log(-tf.math.log(U + eps) + eps)

def gumbel_softmax(temperature, hard):
    def gumbel_softmax_f(x):
        # I'd like to do the gumbel sampling here, but when I do I get
        # zero gradients during back propogation.
        gumbel_softmax_sample = x# + sample_gumbel(tf.shape(x))        
        y = tf.nn.softmax(gumbel_softmax_sample / temperature)

#         y_d = tfp.distributions.RelaxedOneHotCategorical(temperature, probs=x)
#         y = y_d.sample()
        #y = x
        if hard:
            y_hard = tf.one_hot(tf.argmax(y, -1), 4)
            y = tf.stop_gradient(y_hard - y) + y
        return y
    return gumbel_softmax_f #this isn't acutally gumbel

encoder = tf.keras.Sequential([
    layers.Dense(4096, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(0.0000)),
    layers.Dropout(0.2),
    layers.Dense(4096, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(0.0000)),
    layers.Dropout(0.2),
    layers.Dense(OUTPUT_LEN * 4, activation='relu'),
    layers.Reshape([OUTPUT_LEN, 4]),
    #layers.Activation('softmax'),
    layers.Lambda(gumbel_softmax(GUMBEL_SOFTMAX_TEMP, True)),
], name='encoder')

#encoder = tf.keras.models.load_model('/tf/primo/data/models/exp_encoder_8.h5')

# from primo.models.encoder import ClusteringLayer
# import pickle
# with open('/tf/primo/data/clusters_kmeans_64.pkl', 'rb') as f:
#     kmodel = pickle.loads(f.read())
# kmeans = kmodel.cluster_centers_
# encoder = tf.keras.Sequential([
#     ClusteringLayer(kmeans.shape[0], kmeans, input_dim=4096, alpha=25.0),
#     layers.Dense(1024, activation = 'relu'),
#     layers.Dense(OUTPUT_LEN * 4, activation='relu'),
#     layers.Reshape([OUTPUT_LEN, 4]),
#     layers.Activation('softmax'),
# ], name='encoder')

X_triplets = layers.Input([3, 4096])

# Essentially, we started with ]a batch of feature-vector pairs...
# ...And turned them into a pair of feature-vector batches.
X1, X2, X3 = layers.Lambda(lambda X: (X[:,0,:], X[:,1,:], X[:,2,:]))(X_triplets)

distances = layers.Lambda(lambda Xs:
                          (
                              tf.sqrt(tf.reduce_sum(tf.square(Xs[0]-Xs[1]), axis=1)),
                              tf.sqrt(tf.reduce_sum(tf.square(Xs[0]-Xs[2]), axis=1))
                          ))([X1,X2,X3])

# Independently transforms the batches of feature vectors into soft-max encoded DNA sequences.
S1 = encoder(X1)
S2 = encoder(X2)
S3 = encoder(X3)

# Glue them back together! Back into a batch of feature vector pairs.
S_triplets = layers.Lambda(
    lambda Ss: tf.stack(Ss, axis=-1)
)([S1,S2,S3])

# Dimensions: (batch_size x 80 x 4 x 3 ) (i.e. batch size x DNA length x # of nucleotides x 2)

# Swaps dimensions for the predictor, which wants (batch-size x 3 x DNA length x 4)
S_triplets_T = layers.Lambda(lambda S: tf.transpose(S, [0, 3, 1, 2]))(S_triplets)

encoder_trainer = tf.keras.Model(inputs=X_triplets, outputs=S_triplets_T)

print(encoder.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              16781312  
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 80)                327760    
_________________________________________________________________
reshape (Reshape)            (None, 20, 4)             0         
_________________________________________________________________
lambda (Lambda)              (None, 20, 4)             0   

In [144]:
import secrets
import os

def log_multisite_predictor(seq_pairs):
    n_sites = int(seq_pairs.shape[2] / 20)
    # Split into separate sites
    sites = tf.stack(tf.split(seq_pairs, n_sites, axis=2))
    # Apply predictor to sites independently
    scores = tf.map_fn(tf.function(log10_crispr_spec), sites)
    linear_scores = 10**scores
    return tf.experimental.numpy.log10(tf.reduce_sum(linear_scores, axis=0))

def confidence_by_base(sequences):
    """Takes a batch of softmax sequences, and returns vector of average confidence in each position
    
    sequences: Number of exmaple x Number of bases x 4
    
    Returns: 1 x number of bases
    """
    # reduce last dimension to max base number
    return np.mean(np.max(sequences, axis=-1), axis=0)

def entropy_by_base(sequences):
    x = np.sum(sequences, axis=0)
    x /= np.sum(x, axis=1)[:, None]
    entropy = -np.sum(x * np.log(x + 1e-10), axis=1)
    return entropy

def base_distribution(sequences):
    one_hots = one_hot(sequences)
    bases = np.sum(one_hots, axis=0)

def one_hot(nparray, depth = 0, on_value = 1, off_value = 0):
    if depth == 0:
        depth = np.max(nparray) + 1
    assert np.max(nparray) < depth, "the max index of nparray: {} is larger than depth: {}".format(np.max(nparray), depth)
    shape = nparray.shape
    out = np.ones((*shape, depth), dtype=np.float32) * off_value
    indices = []
    for i in range(nparray.ndim):
        tiles = [1] * nparray.ndim
        s = [1] * nparray.ndim
        s[i] = -1
        r = np.arange(shape[i]).reshape(s)
        if i > 0:
            tiles[i-1] = shape[i-1]
            r = np.tile(r, tiles)
        indices.append(r)
    indices.append(nparray)
    out[tuple(indices)] = on_value
    return out

def UniquenessMetric(y_true, y_pred):
    """Use a single batch of inputs to measure sequence uniqueness while training"""
    # Reduce seqs to a single base (e.g. [0.6, 0.2, 0.1, 0.1] -> [0])
    rounded_seqs = K.argmax(y_pred[:, 0, :, :], axis=-1)
    hashed_seqs = tf.reduce_sum(rounded_seqs * (4 ** tf.range(0, 20, dtype=tf.int64)), axis=-1)
    # Get unique ones
    unique_seqs, _ = tf.unique(hashed_seqs)
    return float(len(unique_seqs)) / float(len(y_pred))
    
def ConfidenceMetric(batch):
    """Uses a single batch of inputs to measure sequence uniqueness while training"""
    def confidence(_y_true, y_pred):
        seqs = y_pred[:, 0, :, :]
        # Get the highest probabilty value for each base
        max_seqs = tf.reduce_max(seqs, axis=-1)
        # Take mean along both batch and seqence dimensions
        return tf.reduce_mean(max_seqs)
    return confidence

def RecallMetric(y_true, y_pred):
    Yp = log10_crispr_spec(tf.gather(y_pred, [0, 1], axis=1))
    return tf.reduce_mean(tf.cast(Yp > -2.0, tf.float32))

def NegRecallMetric(y_true, y_pred):
    Yn = log10_crispr_spec(tf.gather(y_pred, [0, 2], axis=1))
    return tf.reduce_mean(tf.cast(Yn > -2.0, tf.float32))

class OutputMonitorCallback(keras.callbacks.Callback):
    """Captures the distribution of bases on each epoch, and saves
    to a variable to allow inspection after training
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.history = []
        _, self.example_features = next(validation_dataset.random_features(100))
        
    def on_epoch_end(self, epoch, logs={}):
        seqs_softmax = encoder.predict(self.example_features)
        seqs_onehot = one_hot(seqs_softmax.argmax(-1))
        
        seqlen = seqs_onehot.shape[1]
        unique_seqs = np.unique(seqs_onehot, axis=0)
        sequence_uniqueness = len(unique_seqs) / len(seqs_onehot)

        confidence = confidence_by_base(seqs_softmax)
        entropy = entropy_by_base(seqs_softmax)
        base_distributions = seqs_onehot.sum(0) / len(seqs_onehot)
        
        self.history.append({
            'confidence': confidence,
            'entropy': entropy,
            'base_distributions': base_distributions,
        })

class EarlyStopCallback(keras.callbacks.Callback):
    """Callback allows clean termination of a training call by
    creating a file in the working directory
    """
    def __init__(self, **kwargs):
        uid = secrets.token_hex(2)
        self.__file = f'/tf/primo/signals/{uid}'
        print(f"Touch {self.__file} to terminate training early")
        super().__init__(**kwargs)


    def on_epoch_end(self, epoch, logs={}):
        if os.path.exists(self.__file):
          print (f"\nStopping after Epoch {epoch}")
          self.model.stop_training = True

class TripletLoss(object):
    def __init__(self, margin):
        self.margin = margin
    
    def __call__(self, y_true, y_pred):
        """
        y_pred is triplets of (anchor, positive, negative), with dimensions
        batch_size x 3 x 20 x 4
        """
#         pos_distance = 1-linear_crispr_spec(-1.8)(tf.gather(y_pred, [0,1], axis=1))
#         neg_distance = linear_crispr_spec(-1.8)(tf.gather(y_pred, [0,2], axis=1))
        
        pos_distance = -log10_crispr_spec(tf.gather(y_pred, [0,1], axis=1))
        neg_distance = log10_crispr_spec(tf.gather(y_pred, [0,2], axis=1))
#         pos_distance = -tfp.math.clip_by_value_preserve_gradient(pos_distance, -3.7, 0)
#         neg_distance = -tfp.math.clip_by_value_preserve_gradient(neg_distance, -3.7, 0)
        
        return tf.maximum(pos_distance, 0.5) + tf.maximum(neg_distance, -3.5)
        #return pos_distance - neg_distance + self.margin
        #return tf.maximum(pos_distance - 2*neg_distance + self.margin, 0.0)
        #return neg_distance - pos_distance

    

In [154]:
### 
# Train with full dataset
####
output_monitor = OutputMonitorCallback()
encoder_trainer.compile(
    tf.keras.optimizers.Adagrad(1e-2),
    TripletLoss(3.6),
    run_eagerly=False,
    metrics=[UniquenessMetric, RecallMetric, NegRecallMetric, ConfidenceMetric(train_inputs)])
history = encoder_trainer.fit(
    train_batch_generator,
    validation_data=val_batch_generator,
    validation_steps=5,
    steps_per_epoch=100,
    epochs=500,
    callbacks=[EarlyStopCallback(), output_monitor],
)

Touch /tf/primo/signals/dcb3 to terminate training early
Epoch 1/500
100/100 [==============================] - 56s 557ms/step - loss: -1.5136 - UniquenessMetric: 0.4885 - RecallMetric: 0.7553 - NegRecallMetric: 0.1841 - confidence: 1.0000 - val_loss: -2.0157 - val_UniquenessMetric: 0.4653 - val_RecallMetric: 0.8600 - val_NegRecallMetric: 0.1280 - val_confidence: 1.0000
Epoch 2/500
100/100 [==============================] - 57s 574ms/step - loss: -1.5373 - UniquenessMetric: 0.4927 - RecallMetric: 0.7486 - NegRecallMetric: 0.1707 - confidence: 1.0000 - val_loss: -1.9150 - val_UniquenessMetric: 0.4400 - val_RecallMetric: 0.8560 - val_NegRecallMetric: 0.1413 - val_confidence: 1.0000
Epoch 3/500
100/100 [==============================] - 70s 706ms/step - loss: -1.5198 - UniquenessMetric: 0.4991 - RecallMetric: 0.7458 - NegRecallMetric: 0.1709 - confidence: 1.0000 - val_loss: -1.9438 - val_UniquenessMetric: 0.4373 - val_RecallMetric: 0.8360 - val_NegRecallMetric: 0.1320 - val_confidence: 1.

Epoch 25/500
100/100 [==============================] - 55s 556ms/step - loss: -1.5094 - UniquenessMetric: 0.5177 - RecallMetric: 0.7459 - NegRecallMetric: 0.1747 - confidence: 1.0000 - val_loss: -1.8838 - val_UniquenessMetric: 0.4720 - val_RecallMetric: 0.8413 - val_NegRecallMetric: 0.1387 - val_confidence: 1.0000
Epoch 26/500
100/100 [==============================] - 54s 550ms/step - loss: -1.5427 - UniquenessMetric: 0.5192 - RecallMetric: 0.7598 - NegRecallMetric: 0.1743 - confidence: 1.0000 - val_loss: -1.7620 - val_UniquenessMetric: 0.4800 - val_RecallMetric: 0.8107 - val_NegRecallMetric: 0.1547 - val_confidence: 1.0000
Epoch 27/500
100/100 [==============================] - 55s 559ms/step - loss: -1.5203 - UniquenessMetric: 0.5151 - RecallMetric: 0.7512 - NegRecallMetric: 0.1713 - confidence: 1.0000 - val_loss: -1.8505 - val_UniquenessMetric: 0.4653 - val_RecallMetric: 0.8347 - val_NegRecallMetric: 0.1493 - val_confidence: 1.0000
Epoch 28/500
100/100 [===========================

Epoch 50/500
100/100 [==============================] - 56s 565ms/step - loss: -1.5441 - UniquenessMetric: 0.5119 - RecallMetric: 0.7603 - NegRecallMetric: 0.1683 - confidence: 1.0000 - val_loss: -1.8487 - val_UniquenessMetric: 0.4747 - val_RecallMetric: 0.8240 - val_NegRecallMetric: 0.1453 - val_confidence: 1.0000
Epoch 51/500
100/100 [==============================] - 57s 575ms/step - loss: -1.5475 - UniquenessMetric: 0.5241 - RecallMetric: 0.7609 - NegRecallMetric: 0.1709 - confidence: 1.0000 - val_loss: -1.9992 - val_UniquenessMetric: 0.5053 - val_RecallMetric: 0.8413 - val_NegRecallMetric: 0.1200 - val_confidence: 1.0000
Epoch 52/500
100/100 [==============================] - 70s 710ms/step - loss: -1.5390 - UniquenessMetric: 0.5295 - RecallMetric: 0.7541 - NegRecallMetric: 0.1711 - confidence: 1.0000 - val_loss: -1.7440 - val_UniquenessMetric: 0.4667 - val_RecallMetric: 0.7973 - val_NegRecallMetric: 0.1653 - val_confidence: 1.0000
Epoch 53/500
100/100 [===========================

Epoch 74/500
100/100 [==============================] - 58s 587ms/step - loss: -1.5644 - UniquenessMetric: 0.5288 - RecallMetric: 0.7619 - NegRecallMetric: 0.1675 - confidence: 1.0000 - val_loss: -1.8277 - val_UniquenessMetric: 0.4907 - val_RecallMetric: 0.8067 - val_NegRecallMetric: 0.1387 - val_confidence: 1.0000
Epoch 75/500
100/100 [==============================] - 58s 582ms/step - loss: -1.5668 - UniquenessMetric: 0.5335 - RecallMetric: 0.7559 - NegRecallMetric: 0.1641 - confidence: 1.0000 - val_loss: -1.9317 - val_UniquenessMetric: 0.4867 - val_RecallMetric: 0.8320 - val_NegRecallMetric: 0.1227 - val_confidence: 1.0000
Epoch 76/500
100/100 [==============================] - 57s 572ms/step - loss: -1.5538 - UniquenessMetric: 0.5200 - RecallMetric: 0.7603 - NegRecallMetric: 0.1665 - confidence: 1.0000 - val_loss: -1.8108 - val_UniquenessMetric: 0.4627 - val_RecallMetric: 0.8160 - val_NegRecallMetric: 0.1413 - val_confidence: 1.0000
Epoch 77/500
100/100 [===========================

Epoch 99/500
100/100 [==============================] - 58s 587ms/step - loss: -1.5669 - UniquenessMetric: 0.5324 - RecallMetric: 0.7624 - NegRecallMetric: 0.1693 - confidence: 1.0000 - val_loss: -1.9121 - val_UniquenessMetric: 0.4693 - val_RecallMetric: 0.8413 - val_NegRecallMetric: 0.1440 - val_confidence: 1.0000
Epoch 100/500
100/100 [==============================] - 59s 592ms/step - loss: -1.5435 - UniquenessMetric: 0.5268 - RecallMetric: 0.7605 - NegRecallMetric: 0.1746 - confidence: 1.0000 - val_loss: -1.9061 - val_UniquenessMetric: 0.4667 - val_RecallMetric: 0.8507 - val_NegRecallMetric: 0.1560 - val_confidence: 1.0000
Epoch 101/500
100/100 [==============================] - 59s 592ms/step - loss: -1.5473 - UniquenessMetric: 0.5277 - RecallMetric: 0.7563 - NegRecallMetric: 0.1690 - confidence: 1.0000 - val_loss: -1.8594 - val_UniquenessMetric: 0.4720 - val_RecallMetric: 0.8347 - val_NegRecallMetric: 0.1560 - val_confidence: 1.0000
Epoch 102/500
100/100 [========================

...

Epoch 291/500
100/100 [==============================] - 57s 573ms/step - loss: -1.5645 - UniquenessMetric: 0.5557 - RecallMetric: 0.7643 - NegRecallMetric: 0.1709 - confidence: 1.0000 - val_loss: -1.8339 - val_UniquenessMetric: 0.5187 - val_RecallMetric: 0.8173 - val_NegRecallMetric: 0.1400 - val_confidence: 1.0000
Epoch 292/500
100/100 [==============================] - 57s 571ms/step - loss: -1.5529 - UniquenessMetric: 0.5628 - RecallMetric: 0.7560 - NegRecallMetric: 0.1648 - confidence: 1.0000 - val_loss: -1.9273 - val_UniquenessMetric: 0.5227 - val_RecallMetric: 0.8240 - val_NegRecallMetric: 0.1253 - val_confidence: 1.0000
Epoch 293/500
100/100 [==============================] - 57s 574ms/step - loss: -1.5430 - UniquenessMetric: 0.5565 - RecallMetric: 0.7577 - NegRecallMetric: 0.1699 - confidence: 1.0000 - val_loss: -1.9516 - val_UniquenessMetric: 0.5160 - val_RecallMetric: 0.8547 - val_NegRecallMetric: 0.1440 - val_confidence: 1.0000
Epoch 294/500
100/100 [=======================

100/100 [==============================] - 71s 720ms/step - loss: -1.5757 - UniquenessMetric: 0.5628 - RecallMetric: 0.7667 - NegRecallMetric: 0.1682 - confidence: 1.0000 - val_loss: -1.7982 - val_UniquenessMetric: 0.5107 - val_RecallMetric: 0.8013 - val_NegRecallMetric: 0.1440 - val_confidence: 1.0000
Epoch 340/500
100/100 [==============================] - 57s 571ms/step - loss: -1.5588 - UniquenessMetric: 0.5584 - RecallMetric: 0.7541 - NegRecallMetric: 0.1641 - confidence: 1.0000 - val_loss: -1.8982 - val_UniquenessMetric: 0.5093 - val_RecallMetric: 0.8240 - val_NegRecallMetric: 0.1293 - val_confidence: 1.0000
Epoch 341/500
100/100 [==============================] - 57s 574ms/step - loss: -1.5627 - UniquenessMetric: 0.5600 - RecallMetric: 0.7484 - NegRecallMetric: 0.1569 - confidence: 1.0000 - val_loss: -1.9995 - val_UniquenessMetric: 0.5240 - val_RecallMetric: 0.8600 - val_NegRecallMetric: 0.1293 - val_confidence: 1.0000
Epoch 342/500
100/100 [==============================] - 58s

Epoch 363/500
100/100 [==============================] - 57s 572ms/step - loss: -1.5506 - UniquenessMetric: 0.5644 - RecallMetric: 0.7549 - NegRecallMetric: 0.1668 - confidence: 1.0000 - val_loss: -1.8473 - val_UniquenessMetric: 0.5453 - val_RecallMetric: 0.8013 - val_NegRecallMetric: 0.1280 - val_confidence: 1.0000
Epoch 364/500
100/100 [==============================] - 57s 578ms/step - loss: -1.5426 - UniquenessMetric: 0.5689 - RecallMetric: 0.7583 - NegRecallMetric: 0.1704 - confidence: 1.0000 - val_loss: -1.8961 - val_UniquenessMetric: 0.5240 - val_RecallMetric: 0.8307 - val_NegRecallMetric: 0.1520 - val_confidence: 1.0000
Epoch 365/500
100/100 [==============================] - 57s 570ms/step - loss: -1.5678 - UniquenessMetric: 0.5655 - RecallMetric: 0.7561 - NegRecallMetric: 0.1615 - confidence: 1.0000 - val_loss: -1.8134 - val_UniquenessMetric: 0.5013 - val_RecallMetric: 0.7867 - val_NegRecallMetric: 0.1200 - val_confidence: 1.0000
Epoch 366/500
100/100 [=======================

Epoch 411/500
100/100 [==============================] - 57s 576ms/step - loss: -1.5351 - UniquenessMetric: 0.5690 - RecallMetric: 0.7511 - NegRecallMetric: 0.1656 - confidence: 1.0000 - val_loss: -2.0424 - val_UniquenessMetric: 0.5080 - val_RecallMetric: 0.8507 - val_NegRecallMetric: 0.1147 - val_confidence: 1.0000
Epoch 412/500
100/100 [==============================] - 58s 587ms/step - loss: -1.5647 - UniquenessMetric: 0.5723 - RecallMetric: 0.7557 - NegRecallMetric: 0.1597 - confidence: 1.0000 - val_loss: -1.9013 - val_UniquenessMetric: 0.5107 - val_RecallMetric: 0.8387 - val_NegRecallMetric: 0.1227 - val_confidence: 1.0000
Epoch 413/500
100/100 [==============================] - 57s 580ms/step - loss: -1.5585 - UniquenessMetric: 0.5678 - RecallMetric: 0.7595 - NegRecallMetric: 0.1726 - confidence: 1.0000 - val_loss: -1.9880 - val_UniquenessMetric: 0.5093 - val_RecallMetric: 0.8453 - val_NegRecallMetric: 0.1200 - val_confidence: 1.0000
Epoch 414/500
100/100 [=======================

Epoch 459/500
100/100 [==============================] - 57s 580ms/step - loss: -1.5476 - UniquenessMetric: 0.5825 - RecallMetric: 0.7553 - NegRecallMetric: 0.1663 - confidence: 1.0000 - val_loss: -1.9118 - val_UniquenessMetric: 0.5160 - val_RecallMetric: 0.8307 - val_NegRecallMetric: 0.1400 - val_confidence: 1.0000
Epoch 460/500
100/100 [==============================] - 58s 584ms/step - loss: -1.5497 - UniquenessMetric: 0.5653 - RecallMetric: 0.7571 - NegRecallMetric: 0.1643 - confidence: 1.0000 - val_loss: -2.0508 - val_UniquenessMetric: 0.5267 - val_RecallMetric: 0.8387 - val_NegRecallMetric: 0.1093 - val_confidence: 1.0000
Epoch 461/500
100/100 [==============================] - 58s 584ms/step - loss: -1.5677 - UniquenessMetric: 0.5697 - RecallMetric: 0.7593 - NegRecallMetric: 0.1643 - confidence: 1.0000 - val_loss: -1.9097 - val_UniquenessMetric: 0.5293 - val_RecallMetric: 0.8373 - val_NegRecallMetric: 0.1373 - val_confidence: 1.0000
Epoch 462/500
100/100 [=======================

In [20]:
# ###
# # Train on a single small batch
# ###
# encoder_trainer.compile(
#     tf.keras.optimizers.Adagrad(1e-2),
#     TripletLoss(0.98),
#     run_eagerly=True,
#     metrics=[UniquenessMetric, ConfidenceMetric(train_inputs)])
# history = encoder_trainer.fit(train_inputs, train_targets, epochs=50)

Epoch 1/50
5/5 [==============================] - 0s 46ms/step - loss: 0.9601 - UniquenessMetric: 1.0000 - confidence: 0.4876
Epoch 2/50
5/5 [==============================] - 0s 46ms/step - loss: 0.9495 - UniquenessMetric: 1.0000 - confidence: 0.4997
Epoch 3/50
5/5 [==============================] - 0s 46ms/step - loss: 0.9466 - UniquenessMetric: 1.0000 - confidence: 0.5193
Epoch 4/50
5/5 [==============================] - 0s 46ms/step - loss: 0.9403 - UniquenessMetric: 1.0000 - confidence: 0.5404
Epoch 5/50
5/5 [==============================] - 0s 47ms/step - loss: 0.9309 - UniquenessMetric: 1.0000 - confidence: 0.5584
Epoch 6/50
5/5 [==============================] - 0s 46ms/step - loss: 0.9171 - UniquenessMetric: 1.0000 - confidence: 0.5807
Epoch 7/50
5/5 [==============================] - 0s 46ms/step - loss: 0.8964 - UniquenessMetric: 1.0000 - confidence: 0.6198
Epoch 8/50
5/5 [==============================] - 0s 46ms/step - loss: 0.8734 - UniquenessMetric: 1.0000 - confidence:

In [156]:
## Save the encoder to a file
#

# encoder.save('/tf/primo/data/models/encoder_1site_triplet_20220203_1.h5')

In [1]:
## Plot training history for last train
#
#

fig = plt.figure()
lines = [k for k in history.history.keys() if not k.startswith('val_')]
lines = [l for l in lines if l != 'confidence']

axes = fig.subplots(int(len(lines)/2), 2)
for i in range(len(lines)):
    label = lines[i]
    ax = axes.flatten()[i]
    val_label = 'val_' + label

    ax.plot(history.history[label], label=label)
    if val_label in history.history:
        ax.plot(history.history[val_label], linestyle=':', label=val_label)
    ax.legend()
    ax.grid()S    

NameError: name 'plt' is not defined

In [26]:
## Tinker with trend of loss

np.polyfit(np.arange(0, 50), history.history['val_loss'][-50:], 1)

array([3.41936320e-05, 2.42410635e-01])